In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file and checks the important fields for our device, and returns True if the data
# is valid and false otherwise.
def validate_dicom_file(filename): 
    print('Validate file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    # check the validity of each DICOM file by analyzing the fields: image position, image type and body part.
    body_part_examined = ds.BodyPartExamined
    modality = ds.Modality
    patient_position = ds.PatientPosition
    
    return body_part_examined == 'CHEST' and modality == 'DX' and patient_position in ['AP', 'PA']
    
    
# This function reads in a .dcm file, and returns a numpy array
# of just the imaging data    
def get_pixel_array(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    return img
    
    
# This function takes the numpy array output by get_pixel_array and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_size): 
    img = img / 255.0  
    proc_img = np.zeros((1024,1024,3))
    
    proc_img[:, :, 0] = img
    proc_img[:, :, 1] = img
    proc_img[:, :, 2] = img
    proc_img = np.resize(proc_img, img_size)
    
    return proc_img


# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        json_savedModel = json_file.read()
    model = tf.keras.models.model_from_json(json_savedModel)
    model.load_weights(weight_path)
    loss = 'binary_crossentropy'
    model.compile(loss=loss)    
    
    return model


# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred_Y = model.predict(img, batch_size = 1, verbose = True)
    prediction = pred_Y > thresh
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.3535353535353536 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    is_valid = validate_dicom_file(i)
    
    if not is_valid:
        continue
    
    img = get_pixel_array(i)
        
    img_proc = preprocess_image(img, IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Validate file test1.dcm ...
Load file test1.dcm ...
1/1 [==============================] - 2s 2s/sample
[[ True]]
Validate file test2.dcm ...
Load file test2.dcm ...
1/1 [==============================] - 0s 26ms/sample
[[ True]]
Validate file test3.dcm ...
Load file test3.dcm ...
1/1 [==============================] - 0s 26ms/sample
[[ True]]
Validate file test4.dcm ...
Validate file test5.dcm ...
Validate file test6.dcm ...
